1. **Корутина для подсчета уникальных слов (0.5б)**  
   Напишите корутину `unique_word_counter`, которая принимает строки (предложения), разбивает их на слова и подсчитывает уникальные слова. При отправке `None` корутина завершает выполнение и печатает итоговое количество уникальных слов.  
   **Пример:**  
   ```python
   coro = unique_word_counter()
   next(coro)
   coro.send("hello world")
   coro.send("hello again")
   coro.send(None)  # -> должно печатать 3, так как слова "hello", "world" и "again" уникальны. Это должно быть только число, подумайте как возвращать значение и обрабатывать результат вызова coro.send(None)
   ```


In [119]:
def unique_word_counter():
    unique_words = set()
    while True:
        sentence = yield
        if sentence is None:
            print(len(unique_words))
            continue
        words = sentence.split()
        unique_words.update(words)

In [120]:
coro = unique_word_counter()
next(coro)
coro.send("hello world")
coro.send("hello again")
coro.send(None)

3


2. **Корутина с ограничением по времени выполнения (1б)**  
   Реализуйте корутину `timed_collector`, которая принимает целые числа и добавляет их к сумме. Если время выполнения корутины с момента ее запуска превышает заданное значение `time_limit` (в секундах), корутина завершается и возвращает текущую сумму. Считайте, что каждый yield значения из корутины занимает время, случайно распределенное между 1 и 3 секундами (реализуйте функцию для этого).
   **Пример использования:**  
   ```python
   coro = timed_collector(time_limit=5)
   next(coro)
   coro.send(10)
   coro.send(20)
   coro.send(30)
   coro.send(40)
   coro.send(50)
   # Через 5 секунд выполнения:
   # Завершение, возвращает текущую сумму: 30, 60 или 100. А может, и 150?
   # Запустите код несколько раз, убедитесь, что возвращаются разные числа!
   # Используйте корректную обработку строк формата coro.send(...)
   ```

In [121]:
import time
import random


def timed_collector(time_limit):
    sum = 0
    start_time = time.time()
    while True:
        current_time = time.time()
        if current_time - start_time >= time_limit:
            return sum

        number = yield
        sum += int(number)
        delay = random.uniform(1, 3)
        print(f"Задержка: {delay:.2f} сек")
        time.sleep(delay)

In [122]:
try:
    coro = timed_collector(time_limit=5)
    next(coro)
    coro.send(10)
    coro.send(20)
    coro.send(30)
    coro.send(40)
    coro.send(50)
except StopIteration as e:
    print(e.value)

Задержка: 1.20 сек
Задержка: 2.02 сек
Задержка: 1.40 сек
Задержка: 1.21 сек
100


3. **Корутина для динамической фильтрации данных с прерыванием (1б)**  
   Напишите корутину `dynamic_filter`, которая принимает числовые значения и фильтрует их по условию, переданному как лямбда-функция. Если отправить значение `None`, корутина завершает выполнение и возвращает список всех значений, прошедших фильтр. Если передать новое условие (новую лямбда-функцию), оно заменяет предыдущее.  
   **Пример использования:**  
   ```python
   coro = dynamic_filter(lambda x: x % 2 == 0)
   next(coro)
   coro.send(10)  # Проходит фильтр
   coro.send(15)  # Не проходит фильтр
   coro.send(lambda x: x > 10)  # Изменение условия фильтрации
   coro.send(15)  # Проходит новый фильтр
   coro.send(None)  # Завершение, возвращает [10, 15]
   ```

In [123]:
def dynamic_filter(condition):
    current_condition = condition
    filtered_values = []

    while True:
        value = yield

        if value is None:
            return filtered_values

        if callable(value):
            current_condition = value
        else:
            if current_condition(value):
                filtered_values.append(value)

In [124]:
try:
    coro = dynamic_filter(lambda x: x % 2 == 0)
    next(coro)
    coro.send(10)  # Проходит фильтр
    coro.send(15)  # Не проходит фильтр
    coro.send(lambda x: x > 10)  # Изменение условия фильтрации
    coro.send(15)  # Проходит новый фильтр
    coro.send(None)  # Завершение, возвращает [10, 15]
except StopIteration as e:
    print(e.value)

[10, 15]


4. **Циклический генератор с обработкой исключений для отдельных заданий (0.5б)**  
   Создайте очередь заданий с помощью `deque`, где каждое задание — генератор, возвращающий числа. Реализуйте корутину `cycle_task_queue`, которая поочередно обрабатывает задания, пока очередь не опустеет. Если задание выбрасывает `StopIteration`, оно должно удаляться из очереди. Если задание выбрасывает `ValueError`, корутина должна просто пропустить это задание и перейти к следующему.  
   **Пример использования:**  
   ```python
   from collections import deque

   def task_1():
       yield 1
       yield 2
       raise ValueError("Task 1 error")

   def task_2():
       yield 3
       yield 4
       yield 5

   tasks = deque([task_1(), task_2()])
   coro = cycle_task_queue(tasks)
   for item in coro:
       print(item)
   ```

In [125]:
from collections import deque


def cycle_task_queue(tasks):
    while tasks:
        task = tasks.popleft()
        try:
            result = next(task)
            yield result
            tasks.append(task)
        except StopIteration:
            continue
        except ValueError as e:
            continue

In [126]:
def task_1():
    yield 1
    yield 2
    raise ValueError("Task 1 error")


def task_2():
    yield 3
    yield 4
    yield 5


tasks = deque([task_1(), task_2()])
coro = cycle_task_queue(tasks)
for item in coro:
    print(item)

1
3
2
4
5


6. **Рекурсивное объединение и фильтрация вложенных данных (3б)**  

У вас есть вложенные структуры данных, состоящие из списков и словарей, которые могут содержать строки, а могут — другие списки и словари. Ваша цель — рекурсивно пройти по всей этой структуре и получить только те строки, которые соответствуют заданной функции фильтрации `filter_func`.

Реализуйте функцию `recursive_extract` — генератор, который рекурсивно обходит вложенные списки и словари, используя `yield from` для делегирования обработки вложенных структур, и возвращает строки.

```python
# Функция фильтрации
def contains_keyword(s):
    return "keyword" in s

# Генератор для рекурсивного обхода и извлечения всех строк
def recursive_extract(data):
    # Проверяет тип data:
    # - Если data — строка, возвращает её
    # - Если data — словарь, рекурсивно обходит все значения словаря
    # - Если data — список, рекурсивно обходит все элементы списка
    # Использует yield from для делегирования обработки вложенных структур
    pass

# Функция для фильтрации строк с использованием yield from
def filtered_strings(data, filter_func):
    for string in recursive_extract(data):
        if filter_func(string):
            yield string

# Вложенная структура данных
nested_data = {
    "messages": [
        "This is a message with keyword.",
        "Just a simple message.",
        {"comments": ["Another keyword here.", "Nothing here."]},
        ["List with keyword.", "Another list entry."]
    ],
    "notes": "This is a single note without the word.",
    "logs": {
        "entries": [
            "Log entry with keyword.",
            {"subentry": "A nested keyword here."}
        ]
    }
}

# Итерируемся по отфильтрованным строкам
for line in filtered_strings(nested_data, contains_keyword):
    print(line)
```

### Ожидаемый вывод

```
This is a message with keyword.
Another keyword here.
List with keyword.
Log entry with keyword.
A nested keyword here.
```

In [127]:
def recursive_extract(data):
    if isinstance(data, str):
        yield data
    elif isinstance(data, dict):
        for value in data.values():
            yield from recursive_extract(value)
    elif isinstance(data, list):
        for item in data:
            yield from recursive_extract(item)

In [128]:
# Функция фильтрации
def contains_keyword(s):
    return "keyword" in s


def filtered_strings(data, filter_func):
    for string in recursive_extract(data):
        if filter_func(string):
            yield string


# Вложенная структура данных
nested_data = {
    "messages": [
        "This is a message with keyword.",
        "Just a simple message.",
        {"comments": ["Another keyword here.", "Nothing here."]},
        ["List with keyword.", "Another list entry."]
    ],
    "notes": "This is a single note without the word.",
    "logs": {
        "entries": [
            "Log entry with keyword.",
            {"subentry": "A nested keyword here."}
        ]
    }
}

# Итерируемся по отфильтрованным строкам
for line in filtered_strings(nested_data, contains_keyword):
    print(line)

This is a message with keyword.
Another keyword here.
List with keyword.
Log entry with keyword.
A nested keyword here.
